<br><br><font color=#008000 ><font size='3'>HM fitting <br><br><font color=#008000 ><font size='3'>

Reference:  
    
    Esser, R., Korn, C. W., Ganzer, F., & Haaker, J. (2021). L-DOPA modulates activity in the vmPFC, nucleus accumbens, and VTA during threat extinction learning in humans. eLife, 10, e65280. https://doi.org/10.7554/eLife.65280

In [1]:
import numpy as np
import pandas as pd
import os

behavdir = '/Users/linxiaomin/Desktop/LINIP/Python/RL_Model/ALL/'
df = pd.read_csv(os.path.join(behavdir,'DEXT_US_expcectancy.csv')) # find original data at [https://osf.io/6tfu3]

In [2]:
df = df[df.columns.drop(list(df.filter(regex='d1')))]      # 删除 day1 'Acquisition'
df = df[df.columns.drop(list(df.filter(regex='d3')))]      # 删除 day3 'retrieval'
df = df[df.columns.drop(list(df.filter(regex='CSP')))]     # 删除 Block gap
df = df[df.columns.drop(list(df.filter(regex='MEAN')))]    # 删除 mean of US expectancy
df = df.drop(columns = 'USintensity')                      # 删除 US强度
df = df.drop(columns = 'USvalenceRat_Cal')                 # 删除 US不愉悦度
df = df.drop(columns = 'ID')                               # 删除 ID
df = df.dropna(axis=0,thresh=8)                            # 删除 未作出反应 > 1/3 的被试

In [3]:
df.head()

,group,d2_UsexpRating_CS_P_B1_1,d2_UsexpRating_CS_P_B1_2,d2_UsexpRating_CS_P_B1_3,d2_UsexpRating_CS_P_B1_4,d2_UsexpRating_CS_P_B1_5,d2_UsexpRating_CS_P_B1_6,d2_UsexpRating_CS_P_B1_7,d2_UsexpRating_CS_P_B1_8,d2_UsexpRating_CS_P_B2_1,...,d2_UsexpRating_CS_M_B2_7,d2_UsexpRating_CS_M_B2_8,d2_UsexpRating_CS_M_B3_1,d2_UsexpRating_CS_M_B3_2,d2_UsexpRating_CS_M_B3_3,d2_UsexpRating_CS_M_B3_4,d2_UsexpRating_CS_M_B3_5,d2_UsexpRating_CS_M_B3_6,d2_UsexpRating_CS_M_B3_7,d2_UsexpRating_CS_M_B3_8
0,2,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
1,1,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# 按group索引把两组分开

group = df['group'].unique()
for i in group:
    split = df[df['group'].isin([i])]
    exec("df%s = split" % i)

In [5]:
# 生成安慰剂组US expectancy数据

df1 = df1.drop(columns = 'group')
data1 = df1.to_numpy(dtype='int8') 
print(data1.shape)

(21, 48)


In [6]:
# 生成L-DOPA组US expectancy数据

df2 = df2.drop(columns = 'group')
data2 = df2.to_numpy(dtype='int8') 
print(data2.shape)

(22, 48)


In [36]:
# 生成一组随机的 CS+/CS(1/0) 序列， - [test]
# 生成received outcome - [ro]

test = np.random.randint(0,2,48)
ro = np.zeros_like(test)
test

array([1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0])

In [56]:
from HM import *
HM?

Init signature: HM(dim, alpha, beta, w)
Docstring:     
Parameters
----------
dim: length of seq (i.e., len(seq))
alpha, beta ,w : params needed to be estimated 


Modules 
----------
lik: likelihood function, return negative log likelihood (NegLL)
fit: using non-linear Nelder-Mead simplex search algorithm to minimize NegLL summed over all trials for each subjects
plot: 有生之年系列
File:           ~/Desktop/LINIP/Python/未命名文件夹/HM/HM.py
Type:           type
Subclasses:     


In [57]:
HM.lik?

Signature: HM.lik(self, resp, seq, ro)
Docstring:
1. softmax function:
    p = e ** (beta*v) / ∑ e ** (beta*v)
        = 1 / (1 + e ** (beta* (v(cs-)-v(cs+)))
        = 1 / (1 + e ** (-beta * v))  # v(CS-) equals 0     
2. update predict reward:
    v += α * η * PE
3. calculate prediction error:
    PE = ro - v
4. update eta: 
    η = w * |PE| + (1-w) * η
5. NegLL
    -∑ y * log (y_hat)
File:      ~/Desktop/LINIP/Python/未命名文件夹/HM/HM.py
Type:      function


In [39]:
# 定义实验模型

test111 = HM(dim=len(test),alpha = 0.5,beta=4,w=0.5)

In [40]:
# 拟合模型 - 
# y: binary US expectancy 
# y-hat: probabilities of choosing

xfit1,BIC1,LL1 = test111.fit(data1,test,ro)
print(xfit1)
print(BIC1)
print(LL1)

[[ 0.64907258 14.          0.        ]
 [ 0.67184905 14.          0.        ]
 [ 0.61819895 13.99982486  0.09490327]
 [ 0.66228005 14.          0.        ]
 [ 0.79069454 14.          1.        ]
 [ 0.66196494 14.          0.        ]
 [ 0.         14.          1.        ]
 [ 0.         14.          0.81944444]
 [ 0.         14.          0.81944444]
 [ 0.64146537 14.          0.43188635]
 [ 0.60719624 14.          0.        ]
 [ 0.37580047 10.21998889  0.46258362]
 [ 0.67192747 14.          0.        ]
 [ 0.60579986 14.          0.        ]
 [ 0.82756582 14.          1.        ]
 [ 0.14411158 14.          0.        ]
 [ 0.35312172  2.93822157  1.        ]
 [ 0.60594687 14.          0.        ]
 [ 0.63433158 13.99999284  0.        ]
 [ 0.17928158 14.          0.        ]
 [ 0.         14.          0.81944444]]
[16.57477974 16.80009525 35.71596456 26.33089653 23.1048017  19.39882298
 11.6136581  11.61376825 11.61376825 22.74236192 34.37552647 15.945302
 15.4138172  41.30361345 19.40885094

In [41]:
xfit2,BIC2,LL2 = test111.fit(data2,test,ro)
print(xfit2)
print(BIC2)
print(LL2)

[[7.41063979e-01 1.40000000e+01 0.00000000e+00]
 [0.00000000e+00 1.40000000e+01 8.19444444e-01]
 [0.00000000e+00 1.40000000e+01 1.00000000e+00]
 [0.00000000e+00 1.40000000e+01 8.19444444e-01]
 [3.34358671e-01 1.40000000e+01 9.50693835e-02]
 [6.71911981e-01 1.40000000e+01 0.00000000e+00]
 [6.22503094e-01 1.40000000e+01 3.66558824e-01]
 [0.00000000e+00 1.40000000e+01 8.19444444e-01]
 [6.71925943e-01 1.40000000e+01 0.00000000e+00]
 [3.73795738e-01 2.48748669e+00 1.00000000e+00]
 [4.83277383e-01 1.39796096e+01 8.35845393e-08]
 [6.34373096e-01 1.40000000e+01 0.00000000e+00]
 [6.06800906e-01 1.39999805e+01 0.00000000e+00]
 [4.39571014e-01 7.22075432e+00 1.00000000e+00]
 [6.31387383e-01 1.39996199e+01 1.79067190e-01]
 [6.06848562e-01 1.40000000e+01 0.00000000e+00]
 [3.53120109e-01 3.98885951e+00 1.00000000e+00]
 [0.00000000e+00 1.40000000e+01 8.19444444e-01]
 [8.82898948e-01 1.40000000e+01 0.00000000e+00]
 [6.71936877e-01 1.40000000e+01 0.00000000e+00]
 [0.00000000e+00 1.40000000e+01 8.194444

# Next step: 拿出BIC & eta & PE & alpha

In [42]:
def get_params(resp,seq,ro,alpha,beta,w):
    import numpy as np

    eta = np.zeros(np.shape(seq)[0]+1,dtype='float')    
    expProb = np.zeros_like(seq,dtype='float')
    valueV = np.zeros_like(seq,dtype='float')
    PEE = np.zeros_like(seq,dtype='float')
    v = np.array([0.25,0.75])
    eta[0] = 1
    
    for t in range(len(seq)):
        
        p = np.exp(beta*v) / np.sum(np.exp(beta*v))
        
        expProb[t] = p[seq[t]]
        
        PE = ro[t]-v[seq[t]]
        
        PEE[t] = PE
        
        v[seq[t]] = v[seq[t]] + alpha * PE * eta[t]
        
        eta[t+1] = w * np.abs(PE) + (1-w) * eta[t]
        
        valueV[t] = v[seq[t]]
        
    _,*eta = eta
    NegLL = -np.sum(resp * np.log(expProb));
    eta = np.array(eta)
    return eta,valueV,PEE,NegLL

In [43]:
g1eta_mean = []
g1valueV_mean = []
g1PEE_mean = []
g1NegLL_all = []

for n, i in enumerate(xfit1):

    exec("g1eta_%s,g1valueV_%s,g1PEE_%s,g1NegLL_%s = get_params(data1[n,:],test,ro,i[0],i[1],i[2])" % (n,n,n,n))
    
    exec ("g1eta_mean.append(np.mean(g1eta_%s))" % (n))
    exec ("g1valueV_mean.append(np.mean(g1valueV_%s))" % (n))
    exec ("g1PEE_mean.append(np.mean(g1PEE_%s))" % (n))
    exec ("g1NegLL_all.append(g1NegLL_%s)" % (n))

g2eta_mean = []
g2valueV_mean = []
g2PEE_mean = []
g2NegLL_all =[]

for n, i in enumerate(xfit2):
    exec("g2eta_%s,g2valueV_%s,g2PEE_%s,g2NegLL_%s = get_params(data2[n,:],test,ro,i[0],i[1],i[2])" % (n,n,n,n))

    exec ("g2eta_mean.append(np.mean(g2eta_%s))" % (n))
    exec ("g2valueV_mean.append(np.mean(g2valueV_%s))" % (n))
    exec ("g2PEE_mean.append(np.mean(g2PEE_%s))" % (n))
    exec ("g2NegLL_all.append(g2NegLL_%s)" % (n))

In [44]:
from scipy import stats
import numpy as np

In [45]:
stats.ttest_ind(BIC1,BIC2)

Ttest_indResult(statistic=-0.04999435817201226, pvalue=0.9603696515124492)

In [46]:
stats.ttest_ind(g1eta_mean,g2eta_mean)

Ttest_indResult(statistic=0.5256890730735138, pvalue=0.6019361251039703)

In [47]:
stats.ttest_ind(g1valueV_mean,g2valueV_mean)

Ttest_indResult(statistic=-0.6080260605794007, pvalue=0.5465237768852431)

In [48]:
stats.ttest_ind(g1PEE_mean,g2PEE_mean)

Ttest_indResult(statistic=0.604818642919674, pvalue=0.5486324492234439)

In [49]:
stats.ttest_ind(g1NegLL_all,g2NegLL_all)

Ttest_indResult(statistic=0.04829917517011331, pvalue=0.9617123281963362)